In [1]:
%load_ext autoreload
import os

os.chdir(os.path.abspath(os.curdir).replace('notebooks',''))

In [52]:
%autoreload 2

from src.utils import files_in_dir
from tqdm import tqdm


In [4]:
ls_files = files_in_dir('./data/arxiv/tar/',['.pdf'])

In [5]:
import openai
import json
# Environment Variables
import os

with open('openai.key','r') as f:
    openai_api_key = f.read()
openai.api_key = openai_api_key
os.environ['OPENAI_API_KEY'] = openai_api_key

In [6]:
from langchain.document_loaders import PyPDFLoader



In [163]:
for pdf_path in tqdm(ls_files[:1]):

    loader = PyPDFLoader(pdf_path)
    pages = loader.load_and_split()

    docs = loader.load()
    ls_text = '\n'.join([d.page_content for d in docs]).split('\n')
    find_ref = [l for l in ls_text if l.lower().startswith('references')]
    if len(find_ref)!=1:
        print(pdf_path, len(find_ref))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.03it/s]


In [165]:
ls_files[0]

'./data/arxiv/tar/2108.11601v2.pdf'

./data/arxiv/tar/2303.05692v1.pdf 0 --> "5. References"

./data/arxiv/tar/2305.11074v1.pdf 0 --> first line of new column

./data/arxiv/tar/2208.04887v2.pdf 0 --> first line of new column


./data/arxiv/tar/2306.05212v1.pdf 3

./data/arxiv/tar/2210.12887v1.pdf 2


In [32]:
from langchain.chains import create_extraction_chain

In [230]:
docs[0]


NameError: name 'doc' is not defined

In [177]:
print(_input.messages[0].content)


You will receive a text extracted from exactly one page of a PDF file, most of the text will be extracted correctly.
However, there will be parts of the text which will be incorrectly extracted or have missing spaces or line breaks.
Your task is to correct such issues from the PDF extraction process and return clean text. 
Respond only with the corrected text. 
Do not add or change the meaning or words in any way except to fix issues from the 

I would expect you to correct issues like:
- Remove line breaks due to PDF formatting where there is no new paragraph
EXAMPLE INPUT: 
Consequently, all these methods adopt an
off-the-shelf metric for retrieval, leading to sub-
optimal performance.
EXAMPLE OUTPUT:
Consequently, all these methods adopt an off-the-shelf metric for retrieval, leading to sub-optimal performance.

- Keep consistent line breaks. Keep new lines or add missing new line when a paragraph ends. Add double line breaks for new sections: 

EXAMPLE INPUT: 
some text
next line 

In [214]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, separators=['\n','\n\n'], chunk_overlap=0)

docs_split = text_splitter.create_documents(['\n'.join([d.page_content for d in docs])])


In [213]:
_ = [print(d.page_content+'\n\n') for d in docs_split[:4]]        

Retrieval Augmented Code Generation and Summarization
Md Rizwan Parvez§, Wasi Uddin Ahmad§, Saikat Chakraborty†
Baishakhi Ray†, Kai-Wei Chang§
§University of California, Los Angeles,†Columbia University
§{rizwan, wasiahmad, kwchang}@cs.ucla.edu,†{saikatc, rayb}@cs.columbia.edu
Abstract
Software developers write a lot of source code
and documentation during software develop-
ment. Intrinsically, developers often recall
parts of source code or code summaries that
they had written in the past while implement-
ing software or documenting them. To mimic
developers’ code or summary generation be-
havior, we propose a retrieval augmented
framework, REDCODER, that retrieves rel-
evant code or summaries from a retrieval
database and provides them as a supplement
to code generation or summarization mod-
els. REDCODER has a couple of uniqueness.
First, it extends the state-of-the-art dense re-
trieval technique to search for relevant code
or summaries. Second, it can work with re-


trieval datab

In [215]:
from rsch_prompts.pdf_clean_prompt import pdf_cleaner_prompt
res = []
for d in tqdm(docs_split):
    _input = pdf_cleaner_prompt.format_prompt(
                    input = d.page_content
                )

    output = llm3(_input.to_messages())
    res.append(output.content)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 58/58 [06:19<00:00,  6.54s/it]


In [223]:
pdf_name = pdf_path.split('/')[-1].replace('.pdf','')

with open(f'./gpt_pp01_{pdf_name}.txt','w') as f:
    f.writelines(res)

In [229]:
print(
"""
as I went to transi-
tion from home if was self-
inflicted
"""
)


as I went to transi-
tion from home if was self-
inflicted



In [211]:
len(_input.to_messages()[0].content)

4426

In [217]:
[print(r) for r in res]

Retrieval Augmented Code Generation and Summarization
Md Rizwan Parvez§, Wasi Uddin Ahmad§, Saikat Chakraborty†
Baishakhi Ray†, Kai-Wei Chang§
§University of California, Los Angeles, †Columbia University
§{rizwan, wasiahmad, kwchang}@cs.ucla.edu, †{saikatc, rayb}@cs.columbia.edu

Abstract
Software developers write a lot of source code and documentation during software development. Intrinsically, developers often recall parts of source code or code summaries that they had written in the past while implementing software or documenting them. To mimic developers’ code or summary generation behavior, we propose a retrieval augmented framework, REDCODER, that retrieves relevant code or summaries from a retrieval database and provides them as a supplement to code generation or summarization models. REDCODER has a couple of uniqueness. First, it extends the state-of-the-art dense retrieval technique to search for relevant code or summaries. Second, it can work with re-
trieval databases that i

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [208]:
print(output.content)

Figure 1: Illustration of our proposed framework REDCODER for code generation. Given an input summary, we first retrieve top-k candidate code (k=1 in this example). We then aggregate them and based on that a generator module generates the target sequence. (for code summarization) from a database. In the second step, a generator processes the retrieved code/summary along with the original input to generate the target output. In this way, REDCODER enhances the generation capability by augmenting the input through retrieval. The two-step process allows us to design a modular and configurable framework for source code and summary generation. Various designs of retriever and generator models can be incorporated into this framework. Existing cross-encoder code retrievers being computationally expensive, their applicability to retrieve from a large database is limited (Humeau et al., 2020). A natural choice would be to use sparse term based retrievers such as TF-IDF or BM25 (Robertson and


In [171]:
schema = {
    "properties": {
        # The title section
        "section_name": {
            "type": "string",
            "description" : "The title of the section. If there is no known title, use value 'CONTINUED'"
        },
        # The contents
        "section_content": {
            "type": "string",
            "description" : "The content of the section"
        },
    },
    "required": ["section_name", "section_content"],
}

In [206]:
chain = create_extraction_chain(schema, llm3)

topics_structured = chain.run(output.content)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


KeyboardInterrupt: 

In [ ]:
topics_structured